# Problem:

Comments belong to the following categories: 
- Toxic
- Severe_toxic
- Obscene
- Threat
- Insult
- Identity_hate 

Each record could have multiple labels on. 

In [ ]:
import pandas as pd

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install contractions

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

import re
import logging
import itertools
import unicodedata
import string
from nltk.corpus import stopwords
import contractions

from bs4 import BeautifulSoup
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip /kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
!unzip /kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
!unzip /kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
!unzip /kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
sample_submission_df = pd.read_csv('sample_submission.csv')
test_labels_df = pd.read_csv('test_labels.csv')

In [ ]:
train_df.head()

### Sample comments corresponding to each category

In [ ]:
for i in ['toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate']:
    print("{} :".format(i.upper()))
    print(train_df.loc[train_df[i]==1, 'comment_text'].sample().values, "\n")

# Text cleaning

Cleaning to remove the following nature of elements from text

- HTML codes
- URLs
- Emojis
- Stopwords
- Punctuations
- Expanding Abbreviations

In [ ]:
train_df.head()

In [ ]:
train_df.describe()

In [ ]:
## Text cleaning

import re

class TextCleaningUtils:
    '''
        This class contains implementations of various text cleaning operations (Static Methods)
    '''

    cleaning_regex_map = {
        'web_links': r'(?i)(?:(?:http(?:s)?:)|(?:www\.))\S+',
#         'special_chars': r'[^a-zA-Z0-9\s\.,!?;:]+',
        'special_chars': r'[^a-zA-Z\s\.,!?;:]+',  ## removing nums
        'redundant_spaces': r'\s\s+',
        'redundant_newlines': r'[\r|\n|\r\n]+',
        'twitter_handles': r'[#@]\S+',
        'punctuations': r'[\.,!?;:]+'
    }
    
    def expand_abbreviations(text):
        text = re.sub(r"he's", "he is", text)
        text = re.sub(r"there's", "there is", text)
        text = re.sub(r"We're", "We are", text)
        text = re.sub(r"That's", "That is", text)
        text = re.sub(r"won't", "will not", text)
        text = re.sub(r"they're", "they are", text)
        text = re.sub(r"Can't", "Cannot", text)
        text = re.sub(r"wasn't", "was not", text)
        text = re.sub(r"don\x89Ûªt", "do not", text)
        text= re.sub(r"aren't", "are not", text)
        text = re.sub(r"isn't", "is not", text)
        text = re.sub(r"What's", "What is", text)
        text = re.sub(r"haven't", "have not", text)
        text = re.sub(r"hasn't", "has not", text)
        text = re.sub(r"There's", "There is", text)
        text = re.sub(r"He's", "He is", text)
        text = re.sub(r"It's", "It is", text)
        text = re.sub(r"You're", "You are", text)
        text = re.sub(r"I'M", "I am", text)
        text = re.sub(r"shouldn't", "should not", text)
        text = re.sub(r"wouldn't", "would not", text)
        text = re.sub(r"couldn't", "could not", text)
        text = re.sub(r"i'm", "I am", text)
        text = re.sub(r"I\x89Ûªm", "I am", text)
        text = re.sub(r"I'm", "I am", text)
        text = re.sub(r"Isn't", "is not", text)
        text = re.sub(r"Here's", "Here is", text)
        text = re.sub(r"you've", "you have", text)
        text = re.sub(r"you\x89Ûªve", "you have", text)
        text = re.sub(r"we're", "we are", text)
        text = re.sub(r"what's", "what is", text)
        text = re.sub(r"couldn't", "could not", text)
        text = re.sub(r"we've", "we have", text)
        text = re.sub(r"it\x89Ûªs", "it is", text)
        text = re.sub(r"doesn\x89Ûªt", "does not", text)
        text = re.sub(r"It\x89Ûªs", "It is", text)
        text = re.sub(r"Here\x89Ûªs", "Here is", text)
        text = re.sub(r"who's", "who is", text)
        text = re.sub(r"I\x89Ûªve", "I have", text)
        text = re.sub(r"y'all", "you all", text)
        text = re.sub(r"can\x89Ûªt", "cannot", text)
        text = re.sub(r"would've", "would have", text)
        text = re.sub(r"it'll", "it will", text)
        text = re.sub(r"we'll", "we will", text)
        text = re.sub(r"wouldn\x89Ûªt", "would not", text)
        text = re.sub(r"We've", "We have", text)
        text = re.sub(r"he'll", "he will", text)
        text = re.sub(r"Y'all", "You all", text)
        text = re.sub(r"Weren't", "Were not", text)
        text = re.sub(r"Didn't", "Did not", text)
        text = re.sub(r"they'll", "they will", text)
        text = re.sub(r"DON'T", "DO NOT", text)
        text = re.sub(r"That\x89Ûªs", "That is", text)
        text = re.sub(r"they've", "they have", text)
        text = re.sub(r"they'd", "they would", text)
        text = re.sub(r"i'd", "I would", text)
        text = re.sub(r"should've", "should have", text)
        text = re.sub(r"You\x89Ûªre", "You are", text)
        text = re.sub(r"where's", "where is", text)
        text = re.sub(r"Don\x89Ûªt", "Do not", text)
        text = re.sub(r"i'll", "I will", text)
        text = re.sub(r"weren't", "were not", text)
        text = re.sub(r"They're", "They are", text)
        text = re.sub(r"Can\x89Ûªt", "Cannot", text)
        text = re.sub(r"you\x89Ûªll", "you will", text)
        text = re.sub(r"I\x89Ûªd", "I would", text)
        text = re.sub(r"let's", "let us", text)
        text = re.sub(r"it's", "it is", text)
        text = re.sub(r"can't", "cannot", text)
        text = re.sub(r"don't", "do not", text)
        text = re.sub(r"you're", "you are", text)
        text = re.sub(r"i've", "I have", text)
        text = re.sub(r"that's", "that is", text)
        text = re.sub(r"i'll", "I will", text)
        text = re.sub(r"doesn't", "does not",text)
        text = re.sub(r"i'd", "I would", text)
        text = re.sub(r"didn't", "did not", text)
        text = re.sub(r"ain't", "am not", text)
        text = re.sub(r"you'll", "you will", text)
        text = re.sub(r"I've", "I have", text)
        text = re.sub(r"Don't", "do not", text)
        text = re.sub(r"I'll", "I will", text)
        text = re.sub(r"I'd", "I would", text)
        text = re.sub(r"Let's", "Let us", text)
        text = re.sub(r"you'd", "You would", text)
        text = re.sub(r"It's", "It is", text)
        text = re.sub(r"Ain't", "am not", text)
        text = re.sub(r"Haven't", "Have not", text)
        text = re.sub(r"Hadn't", "Had not", text)
        text = re.sub(r"Could've", "Could have", text)
        text = re.sub(r"youve", "you have", text)  
        text = re.sub(r"donå«t", "do not", text)  

        return text
    
    def remove_emojis(text):
        emoji_clean= re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        text=emoji_clean.sub(r'',text)
        url_clean= re.compile(r"https://\S+|www\.\S+")
        text=url_clean.sub(r'',text)
        return text
    
#     def remove_punctuations(text):
#         punct_tag=re.compile(r'[^\w\s]')
#         text=punct_tag.sub(r'',text)
#         return text

    @staticmethod
    def clean_text_from_regex(text, text_clean_regex):
        '''
            Follow a particular cleaning expression, provided
            as an input by an user to clean the text.
        '''

        text = text_clean_regex.sub(' ', text).strip()
        return text

    @staticmethod
    def strip_html(text):
        soup = BeautifulSoup(text, "html.parser")
        return soup.get_text()

    @staticmethod
    def remove_special_chars(text):
        '''
            Replace any special character provided as default,
            which is present in the text with space
        '''

        special_chars_regex = re.compile(TextCleaningUtils.cleaning_regex_map['special_chars'])
        text = TextCleaningUtils.clean_text_from_regex(text, special_chars_regex)
        return text

    @staticmethod
    def remove_redundant_spaces(text):
        '''
            Remove any redundant space provided as default,
            that is present in the text.
        '''

        redundant_spaces_regex = re.compile(
            TextCleaningUtils.cleaning_regex_map['redundant_spaces'])
        text = TextCleaningUtils.clean_text_from_regex(text, redundant_spaces_regex)
        return text

    @staticmethod
    def remove_web_links(text):
        '''
            Removes any web link that follows a particular default expression,
            present in the text.
        '''

        web_links_regex = re.compile(TextCleaningUtils.cleaning_regex_map['web_links'])
        text = TextCleaningUtils.clean_text_from_regex(text, web_links_regex)
        return text

    @staticmethod
    def remove_twitter_handles(text):
        '''
            Removes any twitter handle present in the text.
        '''

        twitter_handles_regex = re.compile(TextCleaningUtils.cleaning_regex_map['twitter_handles'])
        text = TextCleaningUtils.clean_text_from_regex(text, twitter_handles_regex)
        return text

    @staticmethod
    def remove_redundant_newlines(text):
        '''
            Removes any redundant new line present in the text.
        '''

        redundant_newlines_regex = re.compile(
            TextCleaningUtils.cleaning_regex_map['redundant_newlines'])
        text = TextCleaningUtils.clean_text_from_regex(text, redundant_newlines_regex)
        return text

    @staticmethod
    def remove_punctuations(text):
        '''
            Removes any punctuation that follows the default expression, in the text.
        '''

        remove_punctuations_regex = re.compile(TextCleaningUtils.cleaning_regex_map['punctuations'])
        text = TextCleaningUtils.clean_text_from_regex(text, remove_punctuations_regex)
        return text

    @staticmethod
    def remove_exaggerated_words(text):
        '''
            Removes any exaggerated word present in the text.
        '''

        return ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))

    @staticmethod
    def replace_multiple_chars(text):
        '''
            Replaces multiple characters present in the text.
        '''

        char_list = ['.', '?', '!', '#', '$', '/', '@', '*', '(', ')', '+']
        final_text = ''
        for i in char_list:
            if i in text:
                pattern = "\\" + i + '{2,}'
                repl_str = i.replace("\\", "")
                text = re.sub(pattern, repl_str, text)
                final_text = ' '.join(text.split())
        return final_text

    @staticmethod
    def replace_sign(text):
        '''
            Replaces any sign with words like & with 'and', in the text.
        '''
        sign_list = {'&': ' and ', '/': ' or ', '\xa0': ' '}
        final_text = ''
        for i in sign_list:
            if i in text:
                text = re.sub(i, sign_list[i], text)
                final_text = ' '.join(text.split())
        return final_text

    @staticmethod
    def remove_accented_char(text):
        text = unicodedata.normalize('NFD', text) \
            .encode('ascii', 'ignore') \
            .decode("utf-8")
        return str(text)

    @staticmethod
    def replace_characters(text, replace_map):
        '''
            Replaces any character custom provided by an user.
        '''

        for char, replace_val in replace_map.items():
            text = text.replace(char, replace_val)
        return text
    
def clean_data(df,col_to_clean):
    df[col_to_clean] = df[col_to_clean].apply(TextCleaningUtils.expand_abbreviations)
    df[col_to_clean] = df[col_to_clean].apply(TextCleaningUtils.remove_emojis)
    df[col_to_clean] = df[col_to_clean].apply(TextCleaningUtils.remove_special_chars)
    df[col_to_clean] = df[col_to_clean].apply(TextCleaningUtils.remove_redundant_spaces)
    df[col_to_clean] = df[col_to_clean].apply(TextCleaningUtils.remove_punctuations)
    df[col_to_clean] = df[col_to_clean].apply(TextCleaningUtils.remove_exaggerated_words)
    df[col_to_clean] = df[col_to_clean].apply(TextCleaningUtils.remove_redundant_newlines)
    df[col_to_clean] = df[col_to_clean].apply(TextCleaningUtils.remove_twitter_handles)
    df[col_to_clean] = df[col_to_clean].apply(TextCleaningUtils.remove_web_links)
    df[col_to_clean] = df[col_to_clean].astype(str)
    df[col_to_clean] = df[col_to_clean].str.lower()
    
    return df

In [ ]:
%%time
train_df = clean_data(train_df, 'comment_text')

In [ ]:
train_df['clean'] = np.where((train_df['toxic']==0) & (train_df['severe_toxic']==0) & (train_df['obscene']==0) & (train_df['threat']==0) & (train_df['insult']==0) & (train_df['identity_hate']==0), 1,0)

In [ ]:
train_df.head(10)

In [ ]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

In [ ]:
train_df['sentence']=train_df['comment_text'].apply(lambda x: sent_tokenize(x))

train_df['tokens']=train_df['comment_text'].apply(lambda x: word_tokenize(x))

In [ ]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))

def get_clean_token_list(words):
    clean_words=[]
    for word in words:
        flag=True
        for letter in word:
            if letter.isdigit() or letter in string.punctuation:
                flag=False
        if flag:
            clean_words.append(word)
    return clean_words

print(get_clean_token_list(["hewy","ho!w","are","yo6u","?dd","111","qwerty"]))

def remove_stopwords(words):
    """
    pass series get series
    """
    filtered_sent=[]
    for word in words:
        if word not in stop_words:
            filtered_sent.append(word)
    return filtered_sent


print(remove_stopwords(["hey","how","are","you","?"]))

In [ ]:
train_df['tokens']=train_df['tokens'].apply(remove_stopwords)

train_df['tokens']=train_df['tokens'].apply(get_clean_token_list)

train_df['tokens'].head()

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
stem = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

def stem_sentence(sentence):
    stem_sentence=[]
    for word in sentence:
        stem_sentence.append(stem.stem(word))
    return stem_sentence

def lemmatize_sentence(sentence):
    lem_sentence=[]
    for word in sentence:
        lem_sentence.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
    return lem_sentence

In [ ]:
train_df['stem_sentence']=train_df['tokens'].apply(lambda x: stem_sentence(x))



train_df['lemm_sentence']=train_df['tokens'].apply(lambda x: lemmatize_sentence(x))



In [ ]:
train_df[['tokens','stem_sentence']].head(10)

In [ ]:
train_df[['tokens','lemm_sentence']].head(10)

In [ ]:
from nltk.probability import FreqDist

In [ ]:
reviews=[word for review in train_df['lemm_sentence'] for word in review]

fdist_reviews = FreqDist(reviews)

fdist_reviews.most_common(20)

In [ ]:
toxic_reviews=train_df[train_df['toxic']==1]['lemm_sentence']

toxic_tokens=[word for review in toxic_reviews for word in review]

fdist_toxic_tokens = FreqDist(toxic_tokens)

fdist_toxic_tokens.most_common(20)

In [ ]:
def get_most_common_tokens(df,label):
    label_reviews=df[df['toxic']==1]['lemm_sentence']
    label_tokens=[word for review in label_reviews for word in review]
    fdist_label_tokens = FreqDist(label_tokens)
    print(fdist_label_tokens.most_common(20))
    return fdist_label_tokens

In [ ]:
train_df.columns

In [ ]:
severe_toxic_reviews=train_df[train_df['severe_toxic']==1]['lemm_sentence']

severe_toxic_tokens=[word for review in severe_toxic_reviews for word in review]

fdist_severe_toxic_tokens = FreqDist(severe_toxic_tokens)

fdist_severe_toxic_tokens.most_common(20)

In [ ]:
import nltk

In [ ]:
train_df.head()

In [ ]:
train_df['pos_tag']=train_df['lemm_sentence'].apply(lambda x: nltk.pos_tag(x))

train_df['pos_tag'].head()

In [ ]:
keep_tags=["JJ","JJR","JJS","RB","RBR","RBS","UH","NN","NNS","NNP","NNPS"]

def filter_tag(pos_list):
    pos_clean_list=[]
    for t in pos_list:
        if t[1] in keep_tags:
            pos_clean_list.append(t[0])
    return pos_clean_list

filter_tag([('asked', 'RB'), ('review', 'NN'), ('scale', 'RBS')])

In [ ]:
train_df['filter_pos_tag']=train_df['pos_tag'].apply(filter_tag)

train_df[['lemm_sentence','filter_pos_tag']].head(10)

In [ ]:
train_df['clean_sentence']=train_df['filter_pos_tag'].apply(lambda x: ' '.join(x))

In [ ]:
train_df.head()

In [ ]:
clean_reviews=train_df[(train_df['toxic']==0)&(train_df['severe_toxic']==0)&(train_df['obscene']==0)&(train_df['threat']==0)&(train_df['insult']==0)&(train_df['identity_hate']==0)]['filter_pos_tag']

clean_tokens=[word for review in clean_reviews for word in review]

fdist_clean_tokens = FreqDist(clean_tokens)

fdist_clean_tokens.most_common(20)

In [ ]:
def get_most_common_tokens(df,label):
    label_reviews=df[df[label]==1]['lemm_sentence']
    label_tokens=[word for review in label_reviews for word in review]
    fdist_label_tokens = FreqDist(label_tokens)
#     print(fdist_label_tokens.most_common(20))
    return fdist_label_tokens

In [ ]:
train_df.columns

In [ ]:
label_list = ['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']

In [ ]:
for label in label_list:
    print(label.upper())
    get_most_common_tokens(train_df,label)

### Closer look at long words 

In [ ]:
train_df.head(20)

In [ ]:
train_df[train_df['clean_sentence'].str.contains("hahahahahahahahahahahahahahahahahahahahahahahahahahahahahahaha")]

In [ ]:
def clean_regex(word):
    lol_word = re.sub(
                r"""(?ix)\b    # assert position at a word boundary
                (?=lol)        # assert that "lol" can be matched here
                \S*            # match any number of characters except whitespace
                (\S+)          # match at least one character (to be repeated later)
                (?<=\blol)     # until we have reached exactly the position after the 1st "lol"
                \1*            # then repeat the preceding character(s) any number of times
                \b             # and ensure that we end up at another word boundary""", 
                "lol", word)
    if lol_word != word:
        return lol_word
                    
    fuck_word = re.sub(
        r"""(?ix)\b    # assert position at a word boundary
        (?=fuck)        # assert that "lol" can be matched here
        \S*            # match any number of characters except whitespace
        (\S+)          # match at least one character (to be repeated later)
        (?<=\bfuck)     # until we have reached exactly the position after the 1st "lol"
        \1*            # then repeat the preceding character(s) any number of times
        \b             # and ensure that we end up at another word boundary""", 
        "fuck", word)
    if fuck_word != word:
        return fuck_word
    
    haha_word = re.sub(
        r"""(?ix)\b    # assert position at a word boundary
        (?=haha)        # assert that "lol" can be matched here
        \S*            # match any number of characters except whitespace
        (\S+)          # match at least one character (to be repeated later)
        (?<=\bhaha)     # until we have reached exactly the position after the 1st "lol"
        \1*            # then repeat the preceding character(s) any number of times
        \b             # and ensure that we end up at another word boundary""", 
        "haha", word)
    if haha_word != word:
        return haha_word
    
    cunt_word = re.sub(
        r"""(?ix)\b    # assert position at a word boundary
        (?=cunt)        # assert that "lol" can be matched here
        \S*            # match any number of characters except whitespace
        (\S+)          # match at least one character (to be repeated later)
        (?<=\bcunt)     # until we have reached exactly the position after the 1st "lol"
        \1*            # then repeat the preceding character(s) any number of times
        \b             # and ensure that we end up at another word boundary""", 
        "cunt", word)
    if cunt_word != word:
        return cunt_word
    
    ty_word = re.sub(
        r"""(?ix)\b    # assert position at a word boundary
        (?=ty)        # assert that "lol" can be matched here
        \S*            # match any number of characters except whitespace
        (\S+)          # match at least one character (to be repeated later)
        (?<=\bty)     # until we have reached exactly the position after the 1st "lol"
        \1*            # then repeat the preceding character(s) any number of times
        \b             # and ensure that we end up at another word boundary""", 
        "ty", word)
    if ty_word != word:
        return ty_word
    return ''

In [ ]:
def remove_repititative_char(sentence):
    clean_sentence = []
    for word in sentence.split():
        if len(word)>23:
            new_word = clean_regex(word)
#             new_word = re.sub(r'([a-z])\1+', r'\1', word)
#             print(word)
#             print(new_word)
            clean_sentence.append(new_word)
        else:
            clean_sentence.append(word)
    return ' '.join(clean_sentence)
    
remove_repititative_char("fuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuck")

In [ ]:
train_df['clean_sentence'] = train_df['clean_sentence'].apply(lambda x: remove_repititative_char(x))

In [ ]:
category_text = train_df[train_df['clean']!=1]['clean_sentence'].values
category_text = " ".join(category_text)
long_len_words = [word for word in category_text.split() if len(word)>23]
print("num of long length words in clean text : {}".format(len(long_len_words)))
long_len_words

In [ ]:
# Word Cloud

## Word Cloud

In [ ]:
from PIL import Image
def display_cloud(label):
    plt.subplots(figsize=(10,10))
    text = train_df[train_df[label]==1]["clean_sentence"].tolist()
    wc = WordCloud(
                          stopwords=STOPWORDS,
                          background_color='black',
                          collocations=False,
                          width=1800,
                          height=800,
                         )
    wc.generate(' '.join(text))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis('off')
    plt.title(label,fontsize=36)
    plt.show()

In [ ]:
for label in label_list:
    display_cloud(label)

## Bi-gram analysis
- Most common bigrams in each category

In [ ]:
def gram_analysis(data,gram):
    stop_words_set = set(stopwords.words('english'))
    tokens=[t for t in data.lower().split(" ") if t!="" if t not in stop_words_set]
    ngrams=zip(*[tokens[i:] for i in range(gram)])
    final_tokens=[" ".join(z) for z in ngrams]
    return final_tokens

In [ ]:
def gram_freq(df, gram, categ_col, text_col):
    category_text = " ".join(df[df[categ_col]==1][text_col].sample(200).values)
    toks = gram_analysis(category_text, gram)
    tok_freq = pd.DataFrame(data=[toks, np.ones(len(toks))]).T.groupby(0).sum().reset_index()
    tok_freq.columns = ['token','frequency']
    tok_freq = tok_freq.sort_values(by='frequency',ascending=False)
    
    plt.figure(figsize=(15,8))
    plt.title("{} most common tokens".format(categ_col))
    sns.barplot(x='token', y='frequency', data=tok_freq.iloc[:30])
    plt.xticks(rotation=90)
    plt.show()
    
    return 

In [ ]:
for i in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'clean']:
    gram_freq(train_df,2, i, 'comment_text')

## Tri-gram analysis
- Most common trigrams in each category

In [ ]:
for i in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'clean']:
    gram_freq(train_df,3, i, 'comment_text')

In [ ]:
train_df.head()

## Vectorization and Embeddings

We will try with TFIDF and Count vectorization strategies and will be going in further sections.

- [TF-IDF Vectorization](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html): This works by applying a logarithmic term to inverse document frequency (IDF) part other than determining the "TF" or term freqency part. The formulation can be shown as follows:

<img src="https://plumbr.io/app/uploads/2016/06/tf-idf.png">

- [Count Vectorization](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html): This is a simpler vectorization technique which relies on frequency of occurence of a particular term in a document or corpus.

A pictorial representation about the way in which vectorization occurs is provided:

<img src="https://www.oreilly.com/library/view/applied-text-analysis/9781491963036/assets/atap_0408.png">

Let us now go ahead and vectorize the corpus and test its dimensionality.

In [ ]:
%%time
tfidf_vect=TfidfVectorizer(stop_words='english',ngram_range=(1,3))
train_tfidf=tfidf_vect.fit_transform(train_df['clean_sentence'].values.tolist())
train_tfidf.shape

In [ ]:
## Outputs from the TF-IDF transformed data
print(train_tfidf)

## Visualizing the  Vector Space

As words and sentences are vectorized, the dimensions of the vector space becomes significantly large to be accomodated in a model. For any computation system it is recommended to keep the dimensions of a tensor (matrix) as small as possible and maintain its regularity.  For tensors with larger dimensions and irregular shapes, it is difficult for the system to perform any operation (matrix /tensor multiplication etc.). Complex operations like tensor differentiation (Jacobian) or numerical approximation is another difficult thing to do for large matrices. The rank plays an important aspect for these operations.

Now, we have to reduce the dimensions ,else the kernel will run out of memory. For this  we wmploy 3 different decomposition techniques:

- [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)
- [SVD](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html)
- [TSNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html)

These algorithms rely on Eigen vector decomposition and Eigen matrices for creating smaller matrices. These reduced matrices are well-fitted to perform any numerical approximation tasks from differentiation to higher order non linear dynamics. [PCA](https://en.wikipedia.org/wiki/Principal_component_analysis) in general is a well known method and forms the base of all decomposition techniques.  Pictorially it operates as follows with the help of orthogonal Eigen vectors:

<img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_ica_vs_pca_thumb.png">


TSNE is a more sophisticated [method](https://lvdmaaten.github.io/publications/papers/AISTATS_2009.pdf)  which uses a non convex optimization along with gradient descent. This is different than Eigen Vector (convex optimization) method of PCA and hence different results may be obtained in different iterations. It is a memory intensive method and is often powerful at the expense of longer execution time.

<img src="https://miro.medium.com/max/685/1*njEd7PiqBW-zW38E23Ho9w.png">


In [ ]:
#Count Vectorization
import matplotlib
import matplotlib.pyplot as plt
def vectorize(data):
    cv=CountVectorizer()
    fit_data_cv=cv.fit_transform(data)
    return fit_data_cv,cv

#Tfidf vectorization from sklearn
def tfidf(data):
    tfidfv=TfidfVectorizer()
    fit_data_tfidf=tfidfv.fit_transform(data)
    return fit_data_cv,tfidfv

def dimen_reduc_plot(test_data,test_label,option):
    tsvd= TruncatedSVD(n_components=2,algorithm="randomized",random_state=42)
    tsne=TSNE(n_components=2,random_state=42) #not recommended instead use PCA
    pca=SparsePCA(n_components=2,random_state=42)
    if(option==1):
        tsvd_result=tsvd.fit_transform(test_data)
        plt.figure(figsize=(10,8))
        colors=['orange','red']
        
        sns.scatterplot(x=tsvd_result[:,0],y=tsvd_result[:,1],hue=test_label        )
        
        plt.show()
        plt.figure(figsize=(10,10))
        plt.scatter(tsvd_result[:,0],tsvd_result[:,1],c=test_label,cmap=matplotlib.colors.ListedColormap(colors))
        color_red=mpatches.Patch(color='red',label='Negative Review')
        color_orange=mpatches.Patch(color='orange',label='Positive Review')
        plt.legend(handles=[color_orange,color_red])
        plt.title("TSVD")
        plt.show()
    if(option==2):
        tsne_result=tsne.fit_transform(test_data)
        plt.figure(figsize=(10,8))
        colors=['orange','red']
        sns.scatterplot(x=tsne_result[:,0],y=tsne_result[:,1],hue=test_label)
        plt.show()
        plt.figure(figsize=(10,10))
        plt.scatter(x=tsne_result[:,0],y=tsne_result[:,1],c=test_label,cmap=matplotlib.colors.ListedColormap(colors))
        color_red=mpatches.Patch(color='red',label='Negative Review')
        color_orange=mpatches.Patch(color='orange',label='Positive Review')
        plt.legend(handles=[color_orange,color_red])
        plt.title("PCA")
        plt.show() 
    if(option==3):
        pca_result=pca.fit_transform(test_data.toarray())
        plt.figure(figsize=(10,8))
        colors=['orange','red']
        sns.scatterplot(x=pca_result[:,0],y=pca_result[:,1],hue=test_label)
        plt.show()
        plt.figure(figsize=(10,10))
        plt.scatter(x=pca_result[:,0],y=pca_result[:,1],c=test_label,cmap=matplotlib.colors.ListedColormap(colors))
        color_red=mpatches.Patch(color='red',label='Negtive Review')
        color_orange=mpatches.Patch(color='orange',label='Positive Review')
        plt.legend(handles=[color_orange,color_red])
        plt.title("TSNE")
        plt.show()
train_data=train_df       
data_vect=train_data['clean_sentence'].values
train_data_cv,cv= vectorize(data_vect)

data_vect_good=count_good['review'].values
target_vect=train_data['Binary'].values
target_data_vect_good=train_df[train_df['sentiment']=='positive']['Binary'].values
data_vect_bad=count_bad['review'].values
target_data_vect_bad=train_df[train_df['sentiment']=='positive']['Binary'].values

real_review_train_data_cv,cv=vectorize(data_vect_good)

print(train_data.head())
dimen_reduc_plot(train_data_cv,target_vect,1)
dimen_reduc_plot(real_review_train_data_cv,target_data_vect_good,1)
dimen_reduc_plot(real_review_train_data_cv,target_data_vect_bad,1)
# dimen_reduc_plot(train_data_cv,target_vect,3)
# dimen_reduc_plot(real_review_train_data_cv,target_data_vect_good,3)
# dimen_reduc_plot(train_data_cv,target_vect,2)
# dimen_reduc_plot(real_review_train_data_cv,target_data_vect_good,2)
